<a href="https://colab.research.google.com/github/hamish-haggerty/AI-hacking/blob/master/SSL/cancer_validation_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cancer_validation_ensemble

> Purpose of this notebook is to explore whether interspersing some BT pretraining makes an ensemble better. 

In [2]:
#| default_exp cancer_validation_ensemble

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [3]:
#| hide

import os
from google.colab import drive

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():

    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')

    os.chdir('cancer-proj')
    
    os.system('pip install .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"') #does this work?

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

Mounted at /content/drive


In [4]:
#| hide
from nbdev.showdoc import *

In [5]:
#| export
from fastai.vision.all import *
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *
from cancer_proj.cancer_maintrain import *

## Load the data

In [6]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

test_eq(x.shape,xtune.shape)

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [7]:
#| hide

aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

## Optionally, display:

In [8]:
#| hide
#show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [9]:
#| hide

#show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

In [10]:
#| export

@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

Need to run a few exploratory experiments. Based on the results, next is to run some systematic experiments, probably with W and B... Or final results...

In [11]:
#| export

@patch
@delegates(Learner.fit_one_cycle)
def encoder_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tuner to use with bt initial weights"
    
    self.freeze() #freeze the resnet
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    #self.unfreeze() #don't unfreeze the resnet. We are fitting training the encoder head + projector
    #self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

    self.unfreeze() #We can unfreeze at the end

## Exploratory experiment: BT initial weights, with a small amount of pretraining. First, let's try updating all of the weights (i.e. the resnet gets updated with BT pretraining). Remember, we need to freeze the pretrained resnet first, and align the encoder-head + projector head.

# We need to edit several of our base functions: Since we have to align the head of the encoder with the projector, we need to edit `create_model`, and define a new bt_splitter: i.e. the splitter needs to freeze the pretrained resnet, and leave the new head_encoder + projector unfrozen.

In [63]:
#| export

class HeadEncoder(nn.Module):
    "Basic nonlinear "
    def __init__(self,resnet_encoder,device='cuda'):
        super().__init__()

        self.resnet_encoder=resnet_encoder

        self.head_encoder = sequential(nn.Linear(2048,2048),nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                               nn.ReLU(inplace=True))
        
        self.device = torch.device(device)
        self.to(self.device)


    def forward(self,x):
        x=self.resnet_encoder(x)
        x=self.head_encoder(x)

        return x

def create_model(which_model,device,ps=8192,n_in=3):
    print('inside create_model')

    #pretrained=True if 'which_model' in ['bt_pretrain', 'supervised_pretrain'] else False

    if which_model == 'bt_pretrain': model = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')
    
    elif which_model == 'no_pretrain': model = resnet50()

    elif which_model == 'supervised_pretrain': model = resnet50(weights='IMAGENET1K_V2')

    #ignore the 'pretrained=False' argument here. Just means we use the weights above 
    #(which themselves are either pretrained or not)
    encoder = get_resnet_encoder(model)
    encoder = Head_Encoder(encoder,device='cpu')

    model = create_barlow_twins_model(encoder, hidden_size=ps,projection_size=ps,nlayers=3)

    if device == 'cuda':
        model.cuda()
        encoder.cuda()


    return model,encoder

bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)

def my_splitter_bt(m):

    return L(sequential(*m.encoder.resnet_encoder),sequential(m.encoder.head_encoder,m.projector)).map(params)

test_eq(len(my_splitter_bt(bt_model)),2)

inside create_model


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [47]:
# #Verify that splitter freezes expected part of model:

# #test : manual. BT

learn = Learner(dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(aug_pipelines,n_in=3,lmb=1/8192,print_augs=False)])
learn.freeze()
print('resnet should be frozen, encoder head + projector unfrozen')
learn.summary()


resnet should be frozen, encoder head + projector unfrozen


BarlowTwinsModel (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                 

## We also need to edit main_train. Now the model is: resnet_encoder -> head_encoder -> projector or linear layer. Also need to edit the splitter fuction for fine tuning.

All that changes here is the definition of our model in `fine_tune`, and we need a new splitter function.

We tried just patching in new def of fine_tune, but since `create_model` defintion changed it fucked things up. 

Feels like this should be some kind of callback, written extensibly enough that you just just "patch" functions etc in...

In [41]:
#| export


class main_train:
    """Instantiate and (optionally) train the encoder. Then fine-tune the supervised model. 
    Outputs metrics on validation data"""

    def __init__(self,
                 dls_train, #used for training BT (if pretrain=True)
                 dls_tune , #used for tuning
                 dls_valid, #used to compute metrics / evaluate results. 
                 xval, #currently `predict_model` below assumes this is entire validation / test data
                 yval,
                 aug_pipelines, #the aug pipeline for self-supervised learning
                 aug_pipelines_tune, #the aug pipeline for supervised learning
                 aug_pipelines_test, #test (or valid) time augmentations 
                 initial_weights, #Which initial weights to use
                 pretrain, #Whether to fit BT
                 num_epochs, #number of BT fit epochs
                 numfit, #number of tune_fit epochs
                 freeze_num_epochs, #How many epochs to freeze body for when training BT
                 freeze_numfit, #How many epochs to freeze body for when fine tuning
                 ps=8192, #projection size
                 n_in=3, #color channels
                 indim=2048, #dimension output of encoder (2048 for resnet50)
                 outdim=9, #number of classes
                 print_report=False, #F1 metrics etc
                 print_plot=False, #ROC curve
                 ):
        store_attr()
        self.vocab = self.dls_valid.vocab
        self.device = 'cuda' if torch.cuda.is_available else 'cpu'

                
                 

                 #Soon we might want to save some models here:

                 #if self.model_type == 'res_proj': test_eq(self.fit_policy,'resnet_fine_tune') #I THINK this is only viable option?
                 #self.encoder_path = f'/content/drive/My Drive/models/baselineencoder_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'
                 #self.tuned_model_path = f'/content/drive/My Drive/models/baselinefinetuned_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'

    @staticmethod
    def fit(learn,fit_type,epochs,freeze_epochs,initial_weights):
        """We can patch in a modification, e.g. if we want subtype of fine_tune:supervised_pretrain to be different
        to fine_tune:bt_pretrain"""

        if fit_type == 'encoder_fine_tune': #i.e. barlow twins

            learn.encoder_fine_tune(epochs,freeze_epochs=freeze_epochs) 

        elif fit_type == 'fine_tune':
            
            #elif initial_weights == 'supervised_pretrain':
            learn.linear_fine_tune(epochs,freeze_epochs=freeze_epochs) 

        else: raise Exception('Fit policy not of expected form')

    def train_encoder(self):
        "create encoder and (optionally, if pretrain=True) train with BT algorithm, according to fit_policy"

        try: #get existing encoder and plonk on new projector
            encoder = self.encoder
            encoder.cpu()
            bt_model = create_barlow_twins_model(encoder, hidden_size=self.ps,projection_size=self.ps,nlayers=3)
            bt_model.cuda()

        except AttributeError: #otherwise, create
            bt_model,encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=self.device)

        if self.pretrain: #train encoder according to fit policy

            learn = Learner(self.dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(self.aug_pipelines,n_in=self.n_in,lmb=1/self.ps,print_augs=False)])
            main_train.fit(learn,fit_type='encoder_fine_tune',
                           epochs=self.num_epochs,freeze_epochs=self.freeze_num_epochs,
                           initial_weights=self.initial_weights
                          )
            
        self.encoder = bt_model.encoder

    def fine_tune(self):
        "fine tune in supervised fashion, according to tune_fit_policy, and get metrics"

        #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)

        try: 
            encoder = self.encoder
        
        except AttributeError:
            _,self.encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=device)

        #model = LM(self.encoder)
        model = sequential(self.encoder,nn.Linear(2048,9))
        
        learn = Learner(self.dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        #debugging
        #learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        main_train.fit(learn,fit_type='fine_tune',
                       epochs=self.numfit,freeze_epochs=self.freeze_numfit,
                       initial_weights=self.initial_weights
                      ) #fine tuning (don't confuse this with fit policy!)
        
        #model.encoder=encoder
        scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=3)
        #metrics dict will have f1 score, auc etc etc
        metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
        auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
        metrics['acc'],metrics['auc_dict'],metrics['scores'],metrics['preds'],metrics['xval'],metrics['yval'] = acc,auc_dict,scores,preds,self.xval,self.yval
  
        #torch.save(model.state_dict(), self.tuned_model_path)
        return metrics #

    def __call__(self):

        self.train_encoder() #train (or extract) the encoder
        metrics = self.fine_tune()
        
        return metrics



We need to define the splitter function for the fine_tune part of main differently as well:

In [38]:
def my_splitter(m):
    print('inside new my_splitter')
    return L(sequential(*m[0].resnet_encoder),sequential(m[0].head_encoder,m[1])).map(params)

In [39]:
# # #Verify that splitter freezes expected part of model, from linear point of view:

bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
model = sequential(encoder,nn.Linear(2048,9))
test_eq(len(my_splitter(model)),2)
test_eq(len(my_splitter_bt(bt_model)),2)

learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
learn.freeze()
print('resnet should be frozen, encoder_head + linear layer unfrozen')
learn.summary()


inside create_model


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main


inside new my_splitter
inside new my_splitter
resnet should be frozen, encoder_head + linear layer unfrozen


Sequential (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                       

## First we need to verify that the head still gives good performance:

In [42]:
#Non default inputs
initial_weights = 'supervised_pretrain'
pretrain=False
numfit=50
num_epochs='na'
freeze_num_epochs = 'na'
freeze_numfit=3

main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
        aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
        initial_weights=initial_weights,pretrain=pretrain,
        num_epochs=num_epochs,numfit=numfit,freeze_num_epochs=freeze_num_epochs,freeze_numfit=freeze_numfit,
        print_report=True,
                 )

metrics = main()

inside create_model
inside new my_splitter


epoch,train_loss,valid_loss,time
0,2.222778,None,00:06
1,2.033058,None,00:06
2,1.833118,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,1.128575,None,00:06
1,1.048449,None,00:06
2,0.974071,None,00:07
3,0.901614,None,00:07
4,0.832174,None,00:06
5,0.764216,None,00:07
6,0.700442,None,00:07
7,0.645286,None,00:06
8,0.597811,None,00:06
9,0.549813,None,00:07


                            precision    recall  f1-score   support

         actinic keratosis       0.47      0.70      0.56        20
      basal cell carcinoma       0.75      0.75      0.75        20
            dermatofibroma       0.78      0.74      0.76        19
                  melanoma       0.67      0.50      0.57        20
                     nevus       0.76      0.80      0.78        20
pigmented benign keratosis       0.71      0.50      0.59        20
      seborrheic keratosis       0.47      0.47      0.47        15
   squamous cell carcinoma       0.65      0.65      0.65        20
           vascular lesion       0.81      0.85      0.83        20

                  accuracy                           0.67       174
                 macro avg       0.67      0.66      0.66       174
              weighted avg       0.68      0.67      0.67       174



Ok, great.

## Check for BT as well:

In [44]:
#Non default inputs
initial_weights = 'bt_pretrain'
pretrain=False
numfit=50
num_epochs='na'
freeze_num_epochs = 'na'
freeze_numfit=3

main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
        aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
        initial_weights=initial_weights,pretrain=pretrain,
        num_epochs=num_epochs,numfit=numfit,freeze_num_epochs=freeze_num_epochs,freeze_numfit=freeze_numfit,
        print_report=True,
                 )

metrics = main()

inside create_model


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


inside new my_splitter


epoch,train_loss,valid_loss,time
0,2.203361,None,00:06
1,1.989363,None,00:07
2,1.733887,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,0.929721,None,00:07
1,0.891595,None,00:07
2,0.826238,None,00:07
3,0.782402,None,00:07
4,0.731069,None,00:06
5,0.686856,None,00:07
6,0.649494,None,00:07
7,0.608818,None,00:07
8,0.573187,None,00:07
9,0.534609,None,00:06


                            precision    recall  f1-score   support

         actinic keratosis       0.57      0.65      0.60        20
      basal cell carcinoma       0.68      0.75      0.71        20
            dermatofibroma       0.78      0.74      0.76        19
                  melanoma       0.45      0.45      0.45        20
                     nevus       0.67      0.60      0.63        20
pigmented benign keratosis       0.67      0.50      0.57        20
      seborrheic keratosis       0.36      0.33      0.34        15
   squamous cell carcinoma       0.50      0.55      0.52        20
           vascular lesion       0.82      0.90      0.86        20

                  accuracy                           0.61       174
                 macro avg       0.61      0.61      0.61       174
              weighted avg       0.62      0.61      0.61       174



Ok, maybe. But might need to compare to just linear head later.

## Exploratory baseline ensembling with a nonlinear head, without pretraining:

Just look at bt weights first:

In [51]:
def run_main_train(initial_weights,num_epochs,freeze_numfit,freeze_num_epochs,pretrain=False,num=5):
    "run main_train num times."

    main_dict = {}
    for i in range(num):

        main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
                aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
                initial_weights=initial_weights,pretrain=pretrain,
                num_epochs=num_epochs,numfit=numfit,freeze_num_epochs=freeze_num_epochs,freeze_numfit=freeze_numfit,
                print_report=True,
                        )
        
        metrics = main()
        main_dict[i] = metrics

    return main_dict
        

In [ ]:
    print(f"With initial_weights={initial_weights}, results are: {[results[initial_weights][i]['acc'] for i in range(len(results[initial_weights]))]}")


In [46]:
initial_weights='bt_pretrain'
main_dict = run_main_train(initial_weights=initial_weights,freeze_numfit=3,num=3)

inside create_model


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


inside new my_splitter


epoch,train_loss,valid_loss,time
0,2.215174,None,00:06
1,1.976193,None,00:07
2,1.725272,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


...

inside new my_splitter


epoch,train_loss,valid_loss,time
0,2.239301,None,00:07
1,1.993383,None,00:06
2,1.728280,None,00:06


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,0.928632,None,00:07
1,0.876730,None,00:07
2,0.828388,None,00:07
3,0.793032,None,00:06
4,0.728941,None,00:06
5,0.695324,None,00:07
6,0.651724,None,00:06
7,0.613493,None,00:06
8,0.581156,None,00:06
9,0.543142,None,00:07


                            precision    recall  f1-score   support

         actinic keratosis       0.52      0.65      0.58        20
      basal cell carcinoma       0.77      0.85      0.81        20
            dermatofibroma       0.73      0.84      0.78        19
                  melanoma       0.33      0.20      0.25        20
                     nevus       0.68      0.65      0.67        20
pigmented benign keratosis       0.50      0.40      0.44        20
      seborrheic keratosis       0.50      0.60      0.55        15
   squamous cell carcinoma       0.57      0.65      0.60        20
           vascular lesion       0.94      0.80      0.86        20

                  accuracy                           0.63       174
                 macro avg       0.62      0.63      0.62       174
              weighted avg       0.62      0.63      0.62       174



In [49]:
from itertools import combinations

print('Results for ensembling within bt weights:')

bt_results = list(main_dict.values())
bt_results = list(combinations(bt_results,2)) #all pairs of results. So for num=3, will be 3
for v in bt_results:

    print(f"\nAcc of first guy in ensemble is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}\n')
    

Results for ensembling within bt weights:

Acc of first guy in ensemble is: 0.6034482717514038
Acc of second guy in ensemble is: 0.5977011322975159
Acc of ensemble is:0.6321839094161987


Acc of first guy in ensemble is: 0.6034482717514038
Acc of second guy in ensemble is: 0.6264367699623108
Acc of ensemble is:0.6149425506591797


Acc of first guy in ensemble is: 0.5977011322975159
Acc of second guy in ensemble is: 0.6264367699623108
Acc of ensemble is:0.5862069129943848



Now, let's look at doing some pretraining. For now, say BT pretrain for 10 epochs. Freezing the projector doesn't make sense remember: we are aligning the random head and projector. But, check above: the backbone encoder is frozen. 

Thinking about this more: training the encoder_head on a frozen backbone will make them less variable.

First, let's try an ensemble with the resnet frozen the whole way through.

Notice that the resnet is kept frozen the whole way through (you can look up above to verify the freeze is working as expected):

In [52]:
#| export
@patch
@delegates(Learner.fit_one_cycle)
def encoder_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tuner to use with bt initial weights"
    
    self.freeze() #freeze the resnet
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    #self.unfreeze() #don't unfreeze the resnet. We are fitting training the encoder head + projector
    #self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

    self.unfreeze() #We can unfreeze at the end


if __name__ == "__main__":
    initial_weights='bt_pretrain'
    num_epochs=10
    freeze_num_epochs=1
    freeze_numfit=3
    pretrain=True

    main_dict = run_main_train(initial_weights,num_epochs,freeze_numfit,freeze_num_epochs,pretrain=pretrain,num=3)


inside create_model


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


epoch,train_loss,valid_loss,time
0,5815.563965,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


epoch,train_loss,valid_loss,time
0,2549.980713,None,00:07
1,2210.193848,None,00:07
2,1975.254028,None,00:07
3,1783.753662,None,00:07
4,1628.290894,None,00:07
5,1510.144531,None,00:07
6,1425.851318,None,00:07
7,1362.262817,None,00:07
8,1307.359619,None,00:07
9,1254.344238,None,00:07


...

epoch,train_loss,valid_loss,time
0,2269.722656,None,00:06
1,2120.265869,None,00:07
2,1907.104736,None,00:07
3,1740.244873,None,00:07
4,1595.790405,None,00:07
5,1480.899536,None,00:07
6,1390.459717,None,00:07
7,1332.623169,None,00:07
8,1269.466675,None,00:07
9,1213.991211,None,00:07


inside new my_splitter


epoch,train_loss,valid_loss,time
0,2.183680,None,00:06
1,1.976876,None,00:07
2,1.735690,None,00:07


epoch,train_loss,valid_loss,time
0,0.918647,None,00:07
1,0.883738,None,00:06
2,0.850651,None,00:06
3,0.817230,None,00:07
4,0.772044,None,00:07
5,0.735650,None,00:06
6,0.687852,None,00:07
7,0.647329,None,00:06
8,0.608528,None,00:07
9,0.571963,None,00:07


                            precision    recall  f1-score   support

         actinic keratosis       0.57      0.65      0.60        20
      basal cell carcinoma       0.62      0.75      0.68        20
            dermatofibroma       0.65      0.79      0.71        19
                  melanoma       0.50      0.45      0.47        20
                     nevus       0.60      0.60      0.60        20
pigmented benign keratosis       0.90      0.45      0.60        20
      seborrheic keratosis       0.59      0.67      0.62        15
   squamous cell carcinoma       0.53      0.50      0.51        20
           vascular lesion       0.85      0.85      0.85        20

                  accuracy                           0.63       174
                 macro avg       0.65      0.63      0.63       174
              weighted avg       0.65      0.63      0.63       174



In [58]:
from itertools import combinations

print('Results for ensembling with bt weights, where we just trained the head')
bt_results = list(main_dict.values())
print([bt_results[i]['acc'] for i in range(len(bt_results))])
bt_results = list(combinations(bt_results,2)) #all pairs of results. So for num=3, will be 3
for v in bt_results:

    print(f"\nAcc of first guy in ensemble is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}\n')
    

Results for ensembling with bt weights, where we just trained the head
[0.5804597735404968, 0.6379310488700867, 0.6321839094161987]

Acc of first guy in ensemble is: 0.5804597735404968
Acc of second guy in ensemble is: 0.6379310488700867
Acc of ensemble is:0.6206896305084229


Acc of first guy in ensemble is: 0.5804597735404968
Acc of second guy in ensemble is: 0.6321839094161987
Acc of ensemble is:0.6321839094161987


Acc of first guy in ensemble is: 0.6379310488700867
Acc of second guy in ensemble is: 0.6321839094161987
Acc of ensemble is:0.6379310488700867



As expected, no gains in ensembling. However, some mild hints that perhaps there are some gains in pretraining the projector? Need to run these prior two experiments for longer (more epochs and runs)

A natural thing to try is to just plonk a projector on the end, train BT as usual (unfrozen encoder). Then plonk a random head on the end and fine tune as usual. The idea here is we will create some variation in the BT weights without destroying them: should increase ensemble performance slightly. Note that training just random heads with BT (as above) will DECREASE the variation in the heads: they have gone from random, to all trained on the same objective. 


To do this, we will need to edit everything again (gah!)

Main points in edit in below cell(s): 

- create_model is as original: resnet + projector
- We need to check bt_splitter and verify is working in cell below
- We need to check the linear splitter as well...
- We need to include encoder_fine_tune, to work as before.
- fine_tune now needs a random encoder_head + linear layer

In [65]:
HeadEncoder??

In [64]:
my_splitter??

In [77]:
#| export

def create_model(which_model,device,ps=8192,n_in=3):
    print('inside create_model')

    #pretrained=True if 'which_model' in ['bt_pretrain', 'supervised_pretrain'] else False

    if which_model == 'bt_pretrain': model = torch.hub.load('facebookresearch/barlowtwins:main', 'resnet50')
    
    elif which_model == 'no_pretrain': model = resnet50()

    elif which_model == 'supervised_pretrain': model = resnet50(weights='IMAGENET1K_V2')

    #ignore the 'pretrained=False' argument here. Just means we use the weights above 
    #(which themselves are either pretrained or not)
    encoder = get_resnet_encoder(model)
    #encoder = HeadEncoder(encoder,device='cpu')

    model = create_barlow_twins_model(encoder, hidden_size=ps,projection_size=ps,nlayers=3)

    if device == 'cuda':
        model.cuda()
        encoder.cuda()


    return model,encoder

class main_train:
    """Instantiate and (optionally) train the encoder. Then fine-tune the supervised model. 
    Outputs metrics on validation data"""

    def __init__(self,
                 dls_train, #used for training BT (if pretrain=True)
                 dls_tune , #used for tuning
                 dls_valid, #used to compute metrics / evaluate results. 
                 xval, #currently `predict_model` below assumes this is entire validation / test data
                 yval,
                 aug_pipelines, #the aug pipeline for self-supervised learning
                 aug_pipelines_tune, #the aug pipeline for supervised learning
                 aug_pipelines_test, #test (or valid) time augmentations 
                 initial_weights, #Which initial weights to use
                 pretrain, #Whether to fit BT
                 num_epochs, #number of BT fit epochs
                 numfit, #number of tune_fit epochs
                 freeze_num_epochs, #How many epochs to freeze body for when training BT
                 freeze_numfit, #How many epochs to freeze body for when fine tuning
                 ps=8192, #projection size
                 n_in=3, #color channels
                 indim=2048, #dimension output of encoder (2048 for resnet50)
                 outdim=9, #number of classes
                 print_report=False, #F1 metrics etc
                 print_plot=False, #ROC curve
                 ):
        store_attr()
        self.vocab = self.dls_valid.vocab
        self.device = 'cuda' if torch.cuda.is_available else 'cpu'

                
                 

                 #Soon we might want to save some models here:

                 #if self.model_type == 'res_proj': test_eq(self.fit_policy,'resnet_fine_tune') #I THINK this is only viable option?
                 #self.encoder_path = f'/content/drive/My Drive/models/baselineencoder_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'
                 #self.tuned_model_path = f'/content/drive/My Drive/models/baselinefinetuned_initial_weights={self.initial_weights}_pretrain={self.pretrain}.pth'

    @staticmethod
    def fit(learn,fit_type,epochs,freeze_epochs,initial_weights):
        """We can patch in a modification, e.g. if we want subtype of fine_tune:supervised_pretrain to be different
        to fine_tune:bt_pretrain"""

        if fit_type == 'encoder_fine_tune': #i.e. barlow twins

            learn.encoder_fine_tune(epochs,freeze_epochs=freeze_epochs) 

        elif fit_type == 'fine_tune':
            
            #elif initial_weights == 'supervised_pretrain':
            learn.linear_fine_tune(epochs,freeze_epochs=freeze_epochs) 

        else: raise Exception('Fit policy not of expected form')

    def train_encoder(self):
        "create encoder and (optionally, if pretrain=True) train with BT algorithm, according to fit_policy"

        try: #get existing encoder and plonk on new projector
            encoder = self.encoder
            encoder.cpu()
            bt_model = create_barlow_twins_model(encoder, hidden_size=self.ps,projection_size=self.ps,nlayers=3)
            bt_model.cuda()

        except AttributeError: #otherwise, create
            bt_model,encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=self.device)

        if self.pretrain: #train encoder according to fit policy

            learn = Learner(self.dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(self.aug_pipelines,n_in=self.n_in,lmb=1/self.ps,print_augs=False)])
            main_train.fit(learn,fit_type='encoder_fine_tune',
                           epochs=self.num_epochs,freeze_epochs=self.freeze_num_epochs,
                           initial_weights=self.initial_weights
                          )
            
        self.encoder = bt_model.encoder

    def fine_tune(self):
        "fine tune in supervised fashion, according to tune_fit_policy, and get metrics"

        #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)

        try: 
            encoder = self.encoder
        
        except AttributeError:
            _,self.encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=device)

        #model = LM(self.encoder)
        encoder = HeadEncoder(self.encoder,device='cuda') #resnet + nonlinear head
        model = sequential(encoder,nn.Linear(2048,9)) #+ linear layer. 
        
        learn = Learner(self.dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        #debugging
        #learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        main_train.fit(learn,fit_type='fine_tune',
                       epochs=self.numfit,freeze_epochs=self.freeze_numfit,
                       initial_weights=self.initial_weights
                      ) #fine tuning (don't confuse this with fit policy!)
        
        #model.encoder=encoder
        scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=3)
        #metrics dict will have f1 score, auc etc etc
        metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
        auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
        metrics['acc'],metrics['auc_dict'],metrics['scores'],metrics['preds'],metrics['xval'],metrics['yval'] = acc,auc_dict,scores,preds,self.xval,self.yval
  
        #torch.save(model.state_dict(), self.tuned_model_path)
        return metrics #

    def __call__(self):

        self.train_encoder() #train (or extract) the encoder
        metrics = self.fine_tune()
        
        return metrics

#The model is now resnet->nonlinear head -> linear layer
def my_splitter(m):

    return L(sequential(*m[0].resnet_encoder),sequential(m[0].head_encoder,m[1])).map(params)


#The model is now just a resnet encoder + a projector
def my_splitter_bt(m):

    return L(sequential(*m.encoder),sequential(m.projector)).map(params)



In [78]:
#| export

@patch
@delegates(Learner.fit_one_cycle)
def encoder_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tuner to use with bt initial weights"
    
    self.freeze() #freeze the resnet
    print('froze resnet')
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze() #Now we want to unfreeze the resnet!
    print('unfroze resnet')
    #self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

    self.unfreeze() #We can unfreeze at the end

In [70]:
#test bt split
bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)

#| hide

#test : manual. BT

learn = Learner(dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(aug_pipelines,n_in=3,lmb=1/8192,print_augs=False)])
learn.freeze()
print('resnet (frozen) + projector')
learn.summary()

inside create_model


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main


body should be frozen, (sans batchnorm) projector unfrozen


BarlowTwinsModel (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                 

In [73]:
#test linear split
bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
encoder = HeadEncoder(encoder,device='cuda') #resnet + nonlinear head
model = sequential(encoder,nn.Linear(2048,9)) #+ linear layer. 
model.cuda()

learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
learn.freeze()
print('resnet (frozen) + unfrozen head and linear layer')
learn.summary()

inside create_model


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main


resnet (frozen) + unfrozen head and linear layer


Sequential (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                       

Go back over your checklist!

##Alright, now our thesis is that pretraining with BT, so long as we don't do it for too long and destroy the representations, will cause diversity and improve ensembling:

(as an aside, we could add a callback/model that implements our ensembling idea IN PROJECTOR SPACE. So the idea is, basically, to push the projectors apart (while the resnet is frozen) as while as aligning it with the resnet, then just train as usual. Make sense!

In [79]:
initial_weights='bt_pretrain'
num_epochs=10
freeze_num_epochs=10
freeze_numfit=3
pretrain=True

main_dict = run_main_train(initial_weights,num_epochs,freeze_numfit,freeze_num_epochs,pretrain=pretrain,num=3)

inside create_model


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main


froze resnet


epoch,train_loss,valid_loss,time
0,4962.552734,None,00:07
1,5110.514160,None,00:07
2,4588.150391,None,00:07
3,3960.423584,None,00:07
4,3521.238037,None,00:07
5,3192.900146,None,00:08
6,2929.714355,None,00:07
7,2705.847900,None,00:07
8,2519.473145,None,00:07
9,2364.213867,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


...

unfroze resnet


epoch,train_loss,valid_loss,time
0,985.852051,None,00:07
1,951.220642,None,00:07
2,977.361267,None,00:08
3,928.955261,None,00:07
4,896.033325,None,00:07
5,856.535522,None,00:07
6,822.716003,None,00:07
7,792.613464,None,00:07
8,773.809204,None,00:08
9,754.545105,None,00:07


epoch,train_loss,valid_loss,time
0,2.171447,None,00:06
1,1.939183,None,00:07
2,1.666695,None,00:07


epoch,train_loss,valid_loss,time
0,0.846983,None,00:07
1,0.824931,None,00:07
2,0.740768,None,00:07
3,0.688299,None,00:07
4,0.644871,None,00:07
5,0.611460,None,00:07
6,0.574170,None,00:07
7,0.535318,None,00:06
8,0.501383,None,00:07
9,0.467709,None,00:07


                            precision    recall  f1-score   support

         actinic keratosis       0.46      0.60      0.52        20
      basal cell carcinoma       0.58      0.70      0.64        20
            dermatofibroma       0.79      0.79      0.79        19
                  melanoma       0.47      0.35      0.40        20
                     nevus       0.65      0.75      0.70        20
pigmented benign keratosis       0.50      0.35      0.41        20
      seborrheic keratosis       0.50      0.53      0.52        15
   squamous cell carcinoma       0.58      0.55      0.56        20
           vascular lesion       0.89      0.80      0.84        20

                  accuracy                           0.60       174
                 macro avg       0.60      0.60      0.60       174
              weighted avg       0.60      0.60      0.60       174



In [80]:
from itertools import combinations

print('Results for ensembling with bt weights, where we trained the usual way (freeze resnet, then unfreeeze)')
bt_results = list(main_dict.values())
print([bt_results[i]['acc'] for i in range(len(bt_results))])
bt_results = list(combinations(bt_results,2)) #all pairs of results. So for num=3, will be 3
for v in bt_results:

    print(f"\nAcc of first guy in ensemble is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}\n')
    

Results for ensembling with bt weights, where we just trained the head
[0.568965494632721, 0.6321839094161987, 0.6034482717514038]

Acc of first guy in ensemble is: 0.568965494632721
Acc of second guy in ensemble is: 0.6321839094161987
Acc of ensemble is:0.6034482717514038


Acc of first guy in ensemble is: 0.568965494632721
Acc of second guy in ensemble is: 0.6034482717514038
Acc of ensemble is:0.6206896305084229


Acc of first guy in ensemble is: 0.6321839094161987
Acc of second guy in ensemble is: 0.6034482717514038
Acc of ensemble is:0.6379310488700867



## Ok, didn't do much. Seems performance is (at least potentially) similar to before. So, a natural next thing to try is to just do the same experiment, but for longer. Also, it makes sense to use a lower base learning rate. Also, it would have been better to make things more extensible...

## In this experiment we edited encoder_fine_tune (essentially lowered the learning rate), and trained for larger number of epochs.

##Lesson: perhaps should have made main_train even more extensible: all hyperparameters it depends on should be passable (even as e.g. dictionaries). Anyway:

In [81]:

@patch
@delegates(Learner.fit_one_cycle)
def encoder_fine_tune(self:Learner, epochs, base_lr=1e-3, freeze_epochs=1, lr_mult=100,
              pct_start=0.3, div=5.0, **kwargs):
    "Fine tuner to use with bt initial weights"
    
    self.freeze() #freeze the resnet
    print('froze resnet')
    self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
    base_lr /= 2
    self.unfreeze() #Now we want to unfreeze the resnet!
    print('unfroze resnet')
    self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
    #self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

    self.unfreeze() #We can unfreeze at the end


initial_weights='bt_pretrain'
num_epochs=50
freeze_num_epochs=10
freeze_numfit=3
pretrain=True

main_dict = run_main_train(initial_weights,num_epochs,freeze_numfit,freeze_num_epochs,pretrain=pretrain,num=3)


from itertools import combinations

print('Results for ensembling with bt weights, where we trained the usual way (freeze resnet, then unfreeeze)')
bt_results = list(main_dict.values())
print([bt_results[i]['acc'] for i in range(len(bt_results))])
bt_results = list(combinations(bt_results,2)) #all pairs of results. So for num=3, will be 3
for v in bt_results:

    print(f"\nAcc of first guy in ensemble is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}\n')
    

inside create_model


Using cache found in /root/.cache/torch/hub/facebookresearch_barlowtwins_main
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


froze resnet


epoch,train_loss,valid_loss,time
0,5421.497070,None,00:07
1,5277.488770,None,00:07
2,5012.328613,None,00:07
3,4495.398926,None,00:07
4,3992.628418,None,00:07
5,3637.159180,None,00:07
6,3326.149902,None,00:07
7,3065.749023,None,00:07
8,2846.154785,None,00:07
9,2657.308838,None,00:07


/usr/local/lib/python3.8/dist-packages/fastprogress/fastprogress.py:73: UserWarning: Your generator is empty.
  warn("Your generator is empty.")


...

unfroze resnet


epoch,train_loss,valid_loss,time
0,1172.511108,None,00:07
1,1141.825562,None,00:07
2,1090.146118,None,00:07
3,1064.508423,None,00:07
4,1025.386230,None,00:07
5,997.650146,None,00:07
6,977.201355,None,00:07
7,953.059021,None,00:08
8,926.502991,None,00:07
9,900.591980,None,00:07


epoch,train_loss,valid_loss,time
0,2.244490,None,00:07
1,1.984441,None,00:07
2,1.725846,None,00:07


epoch,train_loss,valid_loss,time
0,0.895602,None,00:07
1,0.826549,None,00:07
2,0.777876,None,00:07
3,0.741264,None,00:07
4,0.703656,None,00:07
5,0.664868,None,00:07
6,0.627491,None,00:07
7,0.587514,None,00:07
8,0.552690,None,00:07
9,0.517903,None,00:07


                            precision    recall  f1-score   support

         actinic keratosis       0.46      0.60      0.52        20
      basal cell carcinoma       0.62      0.65      0.63        20
            dermatofibroma       0.67      0.84      0.74        19
                  melanoma       0.50      0.35      0.41        20
                     nevus       0.65      0.65      0.65        20
pigmented benign keratosis       0.57      0.40      0.47        20
      seborrheic keratosis       0.47      0.47      0.47        15
   squamous cell carcinoma       0.61      0.55      0.58        20
           vascular lesion       0.77      0.85      0.81        20

                  accuracy                           0.60       174
                 macro avg       0.59      0.60      0.59       174
              weighted avg       0.59      0.60      0.59       174

Results for ensembling with bt weights, where we trained the usual way (freeze resnet, then unfreeeze)
[0.5689654

## WARNING: we changed encoder_fine_tune

In [ ]:
initial_weights_list = ['supervised_pretrain','bt_pretrain']
freeze_numfit_dict = {'supervised_pretrain':3,'bt_pretrain':6}
numfit=50
pretrain=False
results={'supervised_pretrain':None,'bt_pretrain':None}  

for initial_weights in initial_weights_list:
    main_dict = run_main_train(initial_weights=initial_weights,freeze_numfit=freeze_numfit_dict[initial_weights],num=3)
    results[initial_weights] = main_dict #main_dict has result of running main num=3 times.


In [ ]:
def get_ensemble_acc

#Print out result of each run and save: 

In [ ]:
for initial_weights in initial_weights_list:

    print(f"With initial_weights={initial_weights}, results are: {[results[initial_weights][i]['acc'] for i in range(len(results[initial_weights]))]}")


In [ ]:
save_dict_to_gdrive(results,'base_results')

## First observation: results are all very similar, within type of initial weights. Let's first ensemble within models:

In [ ]:
from itertools import combinations

print('Results for ensembling within supervised weights:')

sup_results = list(results['supervised_pretrain'].values())
sup_results = list(combinations(sup_results,2)) #all pairs of results. So for num=3, will be 3
for v in sup_results:

    print(f"Acc of first guy in ensemble is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}')
    
    

Helped on one, not on two. But of course very low sample size (i.e. only 3)

In [ ]:
print('Results for ensembling within bt weights:')

bt_results = list(results['bt_pretrain'].values())
bt_results = list(combinations(bt_results,2)) #all pairs of results. So for num=3, will be 3
for v in sup_results:

    print(f"\nAcc of first guy in ensemble is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}\n')


So, helped on two, not on one; but of course very low sample size. 

Now let's look at ensembling BETWEEN models:

In [ ]:
print('Results for ensembling between models')
import itertools
sup_bt_results = list(itertools.product(results['supervised_pretrain'].values(),results['bt_pretrain'].values()))

for v in sup_bt_results:

    print(f"\nAcc of first guy in ensemble (supervised) is: {v[0]['acc']}")
    print(f"Acc of second guy in ensemble (bt) is: {v[1]['acc']}")
    _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
    print(f'Acc of ensemble is:{acc}\n')


## Ok, results are good enough to continue exploring the ensembling path. Let's re-run the above with a larger sample; and fork the notebook and implement ensembling with some BT pretraining interspersed. 

In [ ]:
#| hide

# #old supervised baseline (with fine tune)

# tem = {0: 0.6724137663841248,
#  1: 0.7126436829566956,
#  2: 0.6724137663841248,
#  3: 0.6321839094161987,
#  4: 0.6896551847457886}

# from statistics import mean
# mean(list(tem.values()))